In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [12]:
dataset = pd.read_csv('sample_data/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
### Divide as dependend and independent features

X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [14]:
### Feature Engineering

geography = pd.get_dummies(X['Geography'], drop_first=True, dtype=int)
gender = pd.get_dummies(X['Gender'], drop_first=True, dtype=int)

In [15]:
X = X.drop(['Geography', 'Gender'], axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,42,2,0.00,1,1,1,101348.88
1,608,41,1,83807.86,1,0,1,112542.58
2,502,42,8,159660.80,3,1,0,113931.57
3,699,39,1,0.00,2,0,0,93826.63
4,850,43,2,125510.82,1,1,1,79084.10


In [16]:
pd.concat([X, geography, gender], axis=1)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [18]:
### Splitting to train and test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [22]:
X_train.shape

(8000, 8)

In [19]:
### Feature Scaling : Features of your dataset are transformed to fall within a specific range or to have specific statistical properties.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU, ReLU
from tensorflow.keras.layers import Dropout

In [21]:
classifier = Sequential()

In [23]:
### Adding the input layer

classifier.add(Dense(units=8, activation='relu'))

In [51]:
### Hidden layer

classifier.add(Dense(units=6, activation='relu'))
classifier.add(Dropout(rate=0.3))

In [52]:
### Hidden layer

classifier.add(Dense(units=5, activation='relu'))
classifier.add(Dropout(rate=0.2))

In [53]:
### Adding the output Layer
classifier.add(Dense(units=1, activation='sigmoid'))

In [54]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [55]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [56]:
### Early Stopping

import tensorflow as tf

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [57]:
model_history = classifier.fit(X_train,
                               y_train,
                               validation_split=0.33, batch_size=10, epochs=1000,
                               callbacks=early_stopping)

Epoch 1/1000
536/536 [==============================] - 6s 6ms/step - loss: 0.5879 - accuracy: 0.7962 - val_loss: 0.5114 - val_accuracy: 0.7955
Epoch 2/1000
536/536 [==============================] - 2s 3ms/step - loss: 0.5135 - accuracy: 0.7962 - val_loss: 0.4878 - val_accuracy: 0.7955
Epoch 3/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.4982 - accuracy: 0.7962 - val_loss: 0.4780 - val_accuracy: 0.7955
Epoch 4/1000
536/536 [==============================] - 2s 3ms/step - loss: 0.4887 - accuracy: 0.7962 - val_loss: 0.4693 - val_accuracy: 0.7955
Epoch 5/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.4831 - accuracy: 0.7962 - val_loss: 0.4614 - val_accuracy: 0.7955
Epoch 6/1000
536/536 [==============================] - 3s 5ms/step - loss: 0.4774 - accuracy: 0.7962 - val_loss: 0.4537 - val_accuracy: 0.7955
Epoch 7/1000
536/536 [==============================] - 3s 5ms/step - loss: 0.4698 - accuracy: 0.7962 - val_loss: 0.4488 - val_accuracy:

In [58]:
model_history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [59]:
### Making the predictions and evaluating the model

y_pred = classifier.predict(X_test)
y_pred = (y_pred >= 0.5)
y_pred

63/63 [==============================] - 0s 2ms/step


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [60]:
### make the confusion matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1595,    0],
       [ 405,    0]])

In [61]:
### Calculate the accuracy

from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
score

0.7975